# 01 Attribute and Pickle the Network
Step 01: Add key attributes to the 'raw' network and create pickles

In [1]:
import os
import sys
import pickle

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import Parameters
from lasso import metcouncil

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

## Remote I/O

In [4]:
lasso_dir = os.path.join('C:/Users', 'david.ory', 'Documents', 'GitHub', 'lasso')
input_dir = os.path.join(lasso_dir, 'examples', 'stpaul')
output_dir = input_dir

## Read Roadway and Transit Networks

In [5]:
link_file = os.path.join(input_dir, 'link.json')
node_file = os.path.join(input_dir, 'node.geojson')
shape_file = os.path.join(input_dir, 'shape.geojson')

roadway_net = RoadwayNetwork.read(link_file = link_file, node_file = node_file, shape_file = shape_file)

2021-03-12 08:37:16, INFO: Reading from following files:
-C:/Users\david.ory\Documents\GitHub\lasso\examples\stpaul\link.json
-C:/Users\david.ory\Documents\GitHub\lasso\examples\stpaul\node.geojson
-C:/Users\david.ory\Documents\GitHub\lasso\examples\stpaul\shape.geojson.
2021-03-12 08:37:16, INFO: Reading from following files:
-C:/Users\david.ory\Documents\GitHub\lasso\examples\stpaul\link.json
-C:/Users\david.ory\Documents\GitHub\lasso\examples\stpaul\node.geojson
-C:/Users\david.ory\Documents\GitHub\lasso\examples\stpaul\shape.geojson.
2021-03-12 08:37:36, INFO: Read 66253 links from C:/Users\david.ory\Documents\GitHub\lasso\examples\stpaul\link.json
2021-03-12 08:37:36, INFO: Read 66253 links from C:/Users\david.ory\Documents\GitHub\lasso\examples\stpaul\link.json
2021-03-12 08:37:36, INFO: Read 17159 nodes from C:/Users\david.ory\Documents\GitHub\lasso\examples\stpaul\node.geojson
2021-03-12 08:37:36, INFO: Read 17159 nodes from C:/Users\david.ory\Documents\GitHub\lasso\examples\st

In [6]:
transit_net = TransitNetwork.read(feed_path = input_dir)

2021-03-12 08:38:05, INFO: Read in transit feed from: C:/Users\david.ory\Documents\GitHub\lasso\examples\stpaul
2021-03-12 08:38:05, INFO: Read in transit feed from: C:/Users\david.ory\Documents\GitHub\lasso\examples\stpaul
2021-03-12 08:38:05, INFO: Removing calendar.txt from transit network config because file not found
2021-03-12 08:38:05, INFO: Removing calendar.txt from transit network config because file not found
2021-03-12 08:38:05, INFO: Removing calendar_dates.txt from transit network config because file not found
2021-03-12 08:38:05, INFO: Removing calendar_dates.txt from transit network config because file not found
2021-03-12 08:38:05, INFO: Removing fare_attributes.txt from transit network config because file not found
2021-03-12 08:38:05, INFO: Removing fare_attributes.txt from transit network config because file not found
2021-03-12 08:38:05, INFO: Removing fare_rules.txt from transit network config because file not found
2021-03-12 08:38:05, INFO: Removing fare_rules.t

## Attribute the Network

In [7]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-03-12 08:38:12, INFO: Lasso base directory set as: C:/Users\david.ory\Documents\GitHub\lasso
2021-03-12 08:38:12, INFO: Lasso base directory set as: C:/Users\david.ory\Documents\GitHub\lasso


In [8]:
roadway_net.links_df = roadway_net.links_df.drop('lanes', axis = 1)
r_net = metcouncil.calculate_number_of_lanes(
        roadway_net=roadway_net,
        parameters=parameters,
)
r_net.links_df.lanes.value_counts()

2021-03-12 08:38:13, INFO: Calculating Number of Lanes as network variable: 'lanes'
2021-03-12 08:38:13, INFO: Calculating Number of Lanes as network variable: 'lanes'
2021-03-12 08:38:13, INFO: Lasso base directory set as: C:/Users\david.ory\Documents\GitHub\lasso
2021-03-12 08:38:13, INFO: Lasso base directory set as: C:/Users\david.ory\Documents\GitHub\lasso
2021-03-12 08:38:13, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2021-03-12 08:38:13, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2021-03-12 08:38:13, INFO: Finished calculating centroid connector variable: centroidconnect
2021-03-12 08:38:13, INFO: Finished calculating centroid connector variable: centroidconnect
2021-03-12 08:38:45, INFO: Finished calculating number of lanes to: lanes
2021-03-12 08:38:45, INFO: Finished calculating number of lanes to: lanes


1.0    9052
2.0     805
4.0      77
3.0      62
5.0      29
6.0       2
Name: lanes, dtype: int64

In [9]:
r_net = metcouncil.calculate_assign_group_and_roadway_class(
        roadway_net=r_net,
        parameters=parameters,
)
r_net.links_df.assign_group.value_counts()

2021-03-12 08:38:46, INFO: Calculating Assignment Group and Roadway Class as network variables: 'assign_group' and 'roadway_class'
2021-03-12 08:38:46, INFO: Calculating Assignment Group and Roadway Class as network variables: 'assign_group' and 'roadway_class'
2021-03-12 08:38:46, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2021-03-12 08:38:46, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2021-03-12 08:40:16, INFO: Finished calculating assignment group variable assign_group and roadway class variable roadway_class
2021-03-12 08:40:16, INFO: Finished calculating assignment group variable assign_group and roadway class variable roadway_class


101    56619
50      4973
9       1856
7       1648
6        787
4        166
2        108
1         76
100       15
11         3
15         2
Name: assign_group, dtype: int64

In [10]:
r_net.links_df.roadway_class.value_counts()

101    56619
50      4973
99      1856
40      1651
30       600
20       297
60       166
10        76
100       15
Name: roadway_class, dtype: int64

In [11]:
r_net.links_df.columns

Index(['model_link_id', 'osm_link_id', 'shstReferenceId', 'shstGeometryId',
       'shape_id', 'u', 'v', 'A', 'B', 'locationReferences', 'distance',
       'roadway', 'name', 'ref', 'bridge', 'tunnel', 'width', 'max_speed',
       'bike_facility', 'drive_access', 'walk_access', 'bike_access',
       'truck_access', 'bus_only', 'rail_only', 'access', 'price',
       'trn_priority', 'ttime_assert', 'geometry', 'centroidconnect', 'lanes',
       'mrcc_id', 'ROUTE_SYS', 'RDWY_CTGY_', 'assign_group', 'roadway_class'],
      dtype='object')

## Create a Scenario and Pickle it

In [12]:
working_scenario = Scenario.create_scenario(base_scenario = {"road_net": r_net, "transit_net": transit_net})

2021-03-12 08:41:00, INFO: Creating Scenario
2021-03-12 08:41:00, INFO: Creating Scenario


In [13]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_full_00.pickle')
pickle.dump(working_scenario, open(working_scenario_filename, 'wb'))